# ERA5-Land Monthly Averaged

* [ERA5-Land Monthly Averaged](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview)
* [Documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation)

## Import dependencies

In [1]:
import os
import cdsapi
import ipywidgets as widgets
from datetime import datetime

from cds_client_util import initialize_cds_client

## Manage Working Directories

In [2]:
# The directories (if doesn't exist) will be created in the Data folder
download_folder = r".\Data\ERA5-land-monthly-means\Download"
working_folder = r".\Data\ERA5-land-monthly-means\Working"
output_folder = r".\Data\ERA5-land-monthly-means\Output"

if not os.path.exists(working_folder):
    os.makedirs(working_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

## CDS API key and authentication

In [3]:
# Use the function to initialize the client
try:
    client = initialize_cds_client("../secrets/.cdsapirc")
    print("CDS API Client initialized successfully!")
except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")

2024-11-21 14:37:23,776 WARNING [2024-11-19T00:00:00] Issue with underlaying infrastructure is now solved. System should gradually recover. Please check updated status [here](https://status.ecmwf.int/)


2024-11-21 14:37:23,776 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.


2024-11-21 14:37:23,776 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2024-11-21 14:37:23,776 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**


2024-11-21 14:37:23,776 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


CDS API Client initialized successfully!


In [4]:
# Alternative: Enter your CDS API key directly
# ------------------------------------------------------
# If you have not created the ".cdsapirc" file or prefer not to use it, 
# you can manually input your CDS API key in this notebook cell.
# This approach is helpful for quick testing or one-off uses but is not recommended 
# for long-term or automated workflows as it requires manual input and may expose your key.
# ------------------------------------------------------

# import getpass

# api_key = getpass.getpass("Enter your CDS API Key:")
# api_url = "https://cds.climate.copernicus.eu/api"
# client = cdsapi.Client(url=api_url, key=api_key)

## Create list of variables and variable group to dynamically select the variable group and variable to download

The list of variables are available at [ERA5-Land Monthly Averaged](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview)

*Please check and modify the predefined variable name if any issue arises with variable name.* 

In [5]:
# Variable Groups
var_group_temperature = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4"]

var_group_lake = [
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    "lake_total_layer_temperature"]

var_group_snow = [
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer"]

var_group_soil_water = [
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4"]

var_group_radiation_and_heat = [
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards"]

var_group_evaporation_and_runoff = [
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation"]

var_group_wind_pressure_and_precipitation = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation"]

var_group_vegetation = [
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"]

# List of variable group
var_group_list = ['var_group_temperature',
                  'var_group_lake',
                  'var_group_snow',
                  'var_group_soil_water',
                  'var_group_radiation_and_heat',
                  'var_group_evaporation_and_runoff',
                  'var_group_wind_pressure_and_precipitation',
                  'var_group_vegetation']

## Define download request parameters to download the desired dataset

In [6]:
selected_variable_group = widgets.Dropdown(
    options=var_group_list,
    value=var_group_list[0],
    description="Select the variable group",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable_group

Dropdown(description='Select the variable group', layout=Layout(width='50%'), options=('var_group_temperature'…

In [7]:
current_variable_group = globals().get(selected_variable_group.value)

selected_variable = widgets.Dropdown(
    options=current_variable_group,
    value=current_variable_group[0],
    description="Select the variable of interest",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable

Dropdown(description='Select the variable of interest', layout=Layout(width='50%'), options=('2m_dewpoint_temp…

In [8]:
# Define the bounding box extents in CRS: WGS84 in this format [lat_max, lon_min, lat_min, lon_max]
# Recommended precision is 0.1 because Horizontal resolution: 0.1° x 0.1°; Native resolution is 9 km.

bbox_wgs84_deutschland = [56.0, 5.8, 47.2, 15.0] # Bouding box for Germany
bbox_wgs84_konstanz = [9.00, 47.60, 9.30, 47.80] # Bounding box for Konstanz, Germany

In [9]:
selected_year = widgets.Dropdown(
    options=[str(year) for year in range(1950, datetime.now().year + 1)],
    value=str(datetime.now().year),
    description="Select the year for downloading data:",
    disabled=False,
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_year

Dropdown(description='Select the year for downloading data:', index=74, layout=Layout(width='50%'), options=('…

In [10]:
selected_year = widgets.Dropdown(
    options=[str(year) for year in range(1950, datetime.now().year + 1)],
    value=str(datetime.now().year),
    description="Select the year for downloading data:",
    disabled=False,
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_year

Dropdown(description='Select the year for downloading data:', index=74, layout=Layout(width='50%'), options=('…

## Define request parameters and download the dataset

In [11]:
# This request downloads 1 year data: 12 months (365 days, 366 days for leap years) and monthly averaged at 00:00.
# The total length of the downloaded dataset is 12*1=12

dataset = "reanalysis-era5-land-monthly-means"
dataset_filename = f"{dataset}_{selected_variable.value}_{selected_year.value}.nc"
dataset_filepath = os.path.join(download_folder,dataset_filename)

request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": selected_variable.value,
    "year": selected_year.value,
    "month": [str(month) for month in range(13)],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": bbox_wgs84_konstanz
}


# Download the dataset with the defined request paremeters
client.retrieve(dataset, request, dataset_filepath)

2024-11-21 14:37:24,158 INFO Request ID is cfe74f91-e8f7-4db2-a505-68257bf5dd66


2024-11-21 14:37:24,257 INFO status has been updated to accepted
